In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment',  None)

In [2]:
customer = pd.read_csv('customer_join.csv')

In [3]:
uselog_months = pd.read_csv('use_log_months.csv')

In [4]:
year_months = list(uselog_months['연월'].unique())

In [5]:
uselog = pd.DataFrame()

In [6]:
uselog_months.head()

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [7]:
for i in range (1,len(year_months)) :
    tmp = uselog_months.loc[uselog_months['연월'] == year_months[i]]
    tmp.rename(columns = {'count' : 'count_0'},inplace = True)
    tmp_before = uselog_months.loc[uselog_months['연월'] == year_months[i-1]]
    del tmp_before['연월']
    tmp_before.rename(columns = {'count' : 'count_1'},inplace = True)
    tmp = pd.merge(tmp, tmp_before, on = 'customer_id', how = 'left')
    uselog = pd.concat([uselog,tmp],ignore_index = True)
uselog.head()

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [8]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer['is_deleted']==1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)) :
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i]-relativedelta(months = 1)

In [9]:
exit_customer['연월'] = pd.to_datetime(exit_customer['exit_date']).dt.strftime('%Y%m')

In [10]:
uselog['연월'] = uselog['연월'].astype(str)

In [11]:
exit_uselog = pd.merge(uselog,exit_customer, on = ['customer_id','연월'], how = 'left')

In [12]:
len(uselog)

33851

In [13]:
exit_uselog.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
exit_uselog = exit_uselog.dropna(subset = ['name'])

In [15]:
len(exit_uselog)

1104

In [16]:
len(exit_uselog['customer_id'].unique())

1104

In [17]:
exit_uselog.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### 지속 회원의 데이터를 작성하자

In [18]:
conti_customer = customer.loc[customer['is_deleted']==0]

In [19]:
conti_uselog = pd.merge(uselog,conti_customer,how = 'left', on = 'customer_id')

In [20]:
len(conti_uselog)

33851

In [21]:
conti_uselog = conti_uselog.dropna(subset=['name'])

In [22]:
len(conti_uselog)

27422

In [23]:
conti_uselog = conti_uselog.sample(frac = 1).reset_index(drop = True)
conti_uselog = conti_uselog.drop_duplicates(subset = 'customer_id')

In [24]:
len(conti_uselog)

2842

#### 지속 회원 데이터와 탈퇴 회원 데이터 결합

In [25]:
predict_data = pd.concat([conti_uselog,exit_uselog], ignore_index = True)

In [26]:
len(predict_data)

3946

In [27]:
predict_data['period'] = 0

In [28]:
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period
0,201901,PL053548,7,6.0,XXXXX,C01,M,2018-02-01,NaN,CA1,...,2_일반,7.583333,7.0,10.0,6.0,1.0,2019-04-30,14.0,NaN,0
1,201903,AS902294,7,6.0,XXXXXX,C02,F,2017-07-01,NaN,CA2,...,0_입회비반액할인,5.833333,6.5,7.0,2.0,1.0,2019-04-30,21.0,NaN,0
2,201901,OA871608,10,4.0,XXXX,C01,F,2017-02-01,NaN,CA1,...,2_일반,5.916667,6.0,10.0,4.0,1.0,2019-04-30,26.0,NaN,0
3,201902,AS984922,7,7.0,XXXXX,C02,M,2017-05-01,NaN,CA1,...,2_일반,5.583333,5.5,7.0,4.0,1.0,2019-04-30,23.0,NaN,0
4,201809,HD707253,9,6.0,XXXXX,C01,M,2018-05-06,NaN,CA2,...,0_입회비반액할인,7.090909,7.0,10.0,5.0,1.0,2019-04-30,11.0,NaN,0


In [29]:
predict_data['now_date'] = pd.to_datetime(predict_data['연월'],format = '%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

In [30]:
for i in range(len(predict_data)) :
    delta = relativedelta(predict_data['now_date'][i],predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years *12 + delta.months)

In [31]:
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201901,PL053548,7,6.0,XXXXX,C01,M,2018-02-01,NaN,CA1,...,7.583333,7.0,10.0,6.0,1.0,2019-04-30,14.0,NaN,11,2019-01-01
1,201903,AS902294,7,6.0,XXXXXX,C02,F,2017-07-01,NaN,CA2,...,5.833333,6.5,7.0,2.0,1.0,2019-04-30,21.0,NaN,20,2019-03-01
2,201901,OA871608,10,4.0,XXXX,C01,F,2017-02-01,NaN,CA1,...,5.916667,6.0,10.0,4.0,1.0,2019-04-30,26.0,NaN,23,2019-01-01
3,201902,AS984922,7,7.0,XXXXX,C02,M,2017-05-01,NaN,CA1,...,5.583333,5.5,7.0,4.0,1.0,2019-04-30,23.0,NaN,21,2019-02-01
4,201809,HD707253,9,6.0,XXXXX,C01,M,2018-05-06,NaN,CA2,...,7.090909,7.0,10.0,5.0,1.0,2019-04-30,11.0,NaN,3,2018-09-01


In [32]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               267
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [33]:
predict_data = predict_data.dropna(subset = ['count_1'])

In [34]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2627
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2627
period                  0
now_date                0
dtype: int64

### 문자열 변수를 처리할 수 있게 가공하자

In [35]:
target_col = ['campaign_name','class_name','gender','count_1','routine_flg','period','is_deleted']

In [36]:
predict_data = predict_data[target_col]

In [37]:
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,2_일반,0_종일,M,6.0,1.0,11,0.0
1,0_입회비반액할인,1_주간,F,6.0,1.0,20,0.0
2,2_일반,0_종일,F,4.0,1.0,23,0.0
3,2_일반,1_주간,M,7.0,1.0,21,0.0
4,0_입회비반액할인,0_종일,M,6.0,1.0,3,0.0


##### 더미변수 만들기

In [38]:
predict_data = pd.get_dummies(predict_data)

In [39]:
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,campaign_name_2_일반,class_name_0_종일,class_name_1_주간,class_name_2_야간,gender_F,gender_M
0,6.0,1.0,11,0.0,0,0,1,1,0,0,0,1
1,6.0,1.0,20,0.0,1,0,0,0,1,0,1,0
2,4.0,1.0,23,0.0,0,0,1,1,0,0,1,0
3,7.0,1.0,21,0.0,0,0,1,0,1,0,0,1
4,6.0,1.0,3,0.0,1,0,0,1,0,0,0,1


In [40]:
del predict_data['campaign_name_2_일반']

In [41]:
del predict_data['class_name_2_야간']

In [42]:
del predict_data['gender_M']

In [43]:
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,class_name_0_종일,class_name_1_주간,gender_F
0,6.0,1.0,11,0.0,0,0,1,0,0
1,6.0,1.0,20,0.0,1,0,0,1,1
2,4.0,1.0,23,0.0,0,0,1,0,1
3,7.0,1.0,21,0.0,0,0,0,1,0
4,6.0,1.0,3,0.0,1,0,1,0,0


### 의사결정 트리를 사용해서 탈퇴 예측 모델을 구축하자

In [44]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

In [45]:
exi = predict_data.loc[predict_data['is_deleted']==1]

In [46]:
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exi))

In [47]:
X = pd.concat([exi,conti],ignore_index = True )

In [48]:
y = X['is_deleted']

In [49]:
del X['is_deleted']

In [50]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

In [51]:
model = DecisionTreeClassifier(random_state=0)

In [52]:
model.fit(X_train,y_train)

DecisionTreeClassifier(random_state=0)

In [53]:
y_test_pred = model.predict(X_test)

In [54]:
print(y_test_pred)

[1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.

In [55]:
results_test = pd.DataFrame({'y_test' : y_test,
                             'y_pred' : y_test_pred})

In [56]:
results_test

,y_test,y_pred
649,1.0,1.0
395,1.0,1.0
720,1.0,1.0
1870,0.0,1.0
918,1.0,1.0
...,...,...
1844,0.0,0.0
710,1.0,1.0
1204,0.0,1.0
2032,0.0,0.0
